In [1]:
import numpy as np
import matplotlib as plt

# Rescorla-Wagner Model of Classical Conditioning

### Step 1. Initializing association weights $w \in \mathbb{R}^n$

In [ ]:
[np.random.uniform() for _ in range(2)]

[0.7099717143963378, 0.8007649148606881]

In [ ]:
# define w
n_feat = 2  # CSes like bell and light 
w = np.random.uniform(high=0.0001, size=2)
print("w:", np.round(w,2))

w: [0. 0.]


### Step 2. Defining value of a Stimulus vector $x$
Given: $w$ weight vector and stimulus vector $x$

In [ ]:
x = [1, 0] # bell, be careful its a python list
x = np.array(x)
l = [w[i]*x[i] for i in range(len(x))]
print(np.round(l,2))
print(np.round(np.sum(l),2))

[0. 0.]
0.0


In [ ]:
# define a function, call it value (receives as input w and x)
def value(w,x):
    sum_value = 0
    for i,wi in enumerate(w):
        sum_value = sum_value + wi*x[i]
    return sum_value

# Testing the value function in this situation: 
print("Value: ",value(w=[1,0.5],x=[1,1]))

Value:  1.5


### Step 3. Defining associative weight update $\Delta w$
$x$: stimulus vector, $r$: reward, $\alpha$: learning rate, n_times: number of updates

In [ ]:
# define the weight update 
def update_rw(w, x, r=1, alpha=0.5, n_times=1):  # times being trials
    x = np.array(x)
    for _ in range(n_times):
        td_error = r - value(w,x) # temporal difference error : reward that you got - predicted one
        w = w + alpha*td_error*x
    return w

### Step 4. Define Classical Conditioning paradigms

In [ ]:
# take the example of forward blocking and define all steps to test results with the model
# 1) initialize weights w and stimulus x
# 2) update weights 10 times for A -> + association
# 3) update weights 10 times for AB -> + association
# 4) print the value of B?

w = np.array([0,0])

# First phase A->+
x = [1,0]
w = update_rw(w,x,r=1, n_times=10)
print("> w after A->+:", np.round(w,4))

# Second AB -> +
x = [1,1]
w = update_rw(w,x,r=1, n_times=10)

print("> w after AB->+", np.round(w,3))
print("Value of B->?", np.round(value(w,[0,1]),2))

> w after A->+: [0.999 0.   ]
> w after AB->+ [1. 0.]
Value of B->? 0.0


In [ ]:
def overexpectation():
    print("overexpectation -------")
    w = np.random.uniform(high=0.001, size=n_feat)
    print("> w init:", np.round(w,2))
    x = [1,0]
    w = update_rw(w, x, n_times=10)
    print("> w after A->+:", np.round(w,2))
    x = [0,1]
    w = update_rw(w, x, n_times=10)
    print("> w after B->+:", np.round(w,2))
    print("Value of B->?", np.round(value(w,[0,1]),2))
    
    w = update_rw(w,[1,1], n_times=10)
    print("> w after AB->+", np.round(w,2))
    
    print("Value of B->?", np.round(value(w,[0,1]),2))

    
overexpectation()

overexpectation -------
> w init: [0. 0.]
> w after A->+: [1. 0.]
> w after B->+: [1. 1.]
Value of B->? 1.0
> w after AB->+ [0.5 0.5]
Value of B->? 0.5


In [ ]:
def overshadowing():
    print("overshadowing -------")
    w = np.random.uniform(high=0.001, size=n_feat)
    print("> w init:", np.round(w,2))
    w = update_rw(w,[1,1], n_times=10)
    print("> w after AB->+", np.round(w,2))
    print("Value of B->?", np.round(value(w,[0,1]),2))

    
overshadowing()

overshadowing -------
> w init: [0. 0.]
> w after AB->+ [0.5 0.5]
Value of B->? 0.5


# Temporal Difference Learning

### Solution 1. Future discounted reward prediction within R&W 
Function approximation $V_t = w^\top \phi(s_t)$ and Indicator features $x_t = \phi(s_t) \in \mathbb{R}^n$ sets a 1 at the corresponding state index (one-hot encoding): $x_t(t) = 1$  

In [ ]:
# define the weight update 
def update_td(w, x, x_next, r=1, alpha=0.5, gamma=0.8, n_times=1):  # times being trials
    x = np.array(x)
    for _ in range(n_times):
        td_error = r + gamma*value(w, x_next) - value(w,x)
        w = w + alpha*td_error*x
    return w

Second Order Conditioning is now captured:

In [ ]:
# Our aim is to test Second Order Conditioning: 
# B->+,   A->B,   A->?
w = [0,0]

A = [1,0]
B = [0,1]

# B->+
w = update_td(w, x=B, x_next=[0,0], r=1, n_times=10)  # with a dummy next state [0,0]
print("w after B->+", w)

# A->B
w = update_td(w, x=A, x_next=B, r=0, gamma=.8, n_times=100)
print("w after A->B", w)

# Value of A after training?
print("Value of A->?", np.round(value(w,A),2))   # Now second order conditioning works!


w after B->+ [0.         0.99902344]
w after A->B [0.79921875 0.99902344]
Value of A->? 0.8


### Solution 2. Global states, no feature approximation

In [3]:
# initializations
alpha=0.5
gamma=0.8         # gamma of 0 brings us back to R&W model
t=0               # will index time steps, 0,1,2
R = [0,1,0]       # defines the reward at time t:  corresponds to A,B,dummy next state  
S = [0,1,2]       # states are represented as integers A=0, B=1
A,B,C = 0,1,2
V = np.zeros(3)   # V[0] = value of state 0: an array used to store "value" of each state: no association weight vector.  

In [4]:
# B->+
t = 1
for _ in range(10): 
    td_error = R[t] + gamma*V[t+1] - V[t] 
    V[t] = V[t] + alpha*td_error

print("Value of B after B->+ (V[B]):", V[B])

# A->B
t = 0
for _ in range(10): 
    td_error = R[t] + gamma*V[t+1] - V[t] 
    V[t] = V[t] + alpha*td_error

print("Value of A after A->B (V[A]):", V[A])


Value of B after B->+ (V[B]): 0.9990234375
Value of A after A->B (V[A]): 0.7984382629394532


## Kalman TD (Gershman)

In [49]:
N,D=5,2
X = np.zeros([N,D])
np.vstack((X,np.zeros([1,D])))
np.identity(D)

A = np.array([1,0])
B = np.array([0,1])
A@B

0

In [252]:
# c - prior variance (default: 1)
# s - observation noise variance (default: 1)
# q - transition noise variance (default: 0.01)

def ktd(X, R, gamma=0.9, c=1, s=1, q=0.005):  
    N,D = np.array(X).shape                       # N - time steps ; D - feature dimensions 
    w = np.zeros(D)                               # weights
    X = np.vstack((np.array(X),np.zeros([1,D])))  # add feature vector at end for dummy X[i+1,:]
    C = c*np.identity(D)
    Q = q*np.identity(D)
    
    print("---------------  KTD:", N,D, "\n")

    for i in range(N):
        print("i:", i, "of", N, "--------------------------------")

        h = X[i,:] - gamma*X[i+1,:]                # temporal difference features: a vector Dx1
        V = X[i,:]@w                               # value estimate: a real
        rhat = h@w                                 # reward prediction: a real
        dt = R[i] - rhat                           # prediction error: a real
        C = C + Q                                  # a priori covariance: a matrix DxD
        P = np.matmul(h,C)@h + s                   # residual covariance: a real
        K = np.matmul(C, h.reshape([D,1]) )/P      # Kalman gain: a vector Dx1
        w = w + K.flatten()*dt                     # weight update: a vector Dx1
        C = C - K*np.matmul(h.reshape([D,1]).T,C)  # posterior covariance update

        print("rhat:",rhat)
        print("dt:",dt)
        print("P:",P)
        print("K:",K.flatten().tolist())
        print("w:", w.tolist())
        print("C:", C.tolist())
        print("")
        
    return w, C

In [251]:
A = [1,0]
B = [0,1]
AB = [1,1]
C = [0,0]

t = 10
X = [AB]*t + [A]*t
R = [1]*t + [1]*t

print("X:",X, "  ", np.array(X).shape)
print("R:",R, "\n")

w,C = ktd(X,R, gamma=0)

X: [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]    (20, 2)
R: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

KTD: 20 2 ---------------------------------------------- 

i: 0 of 20 --------------------------------
rhat: 0.0
dt: 1.0
P: 3.01
K: [0.33388704318936874, 0.33388704318936874]
w: [0.33388704318936874, 0.33388704318936874]
C: [[0.6694435215946843, -0.33555647840531555], [-0.33555647840531555, 0.6694435215946843]]

i: 1 of 20 --------------------------------
rhat: 0.6677740863787375
dt: 0.3322259136212625
P: 1.6777740863787376
K: [0.20198609928516267, 0.20198609928516267]
w: [0.40099205956317696, 0.40099205956317696]
C: [[0.6059930496425813, -0.4040069503574186], [-0.4040069503574186, 0.6059930496425813]]

i: 2 of 20 --------------------------------
rhat: 0.8019841191263539
dt: 0.19801588087364608
P: 1.4139721985703253
K: [0.14638625815588693, 0.146386